In [ ]:
import pandas as pd
import numpy as np
import gzip
import json
import sys
import seaborn as sns
import ast
import warnings
warnings.filterwarnings('ignore')

from sklearn.decomposition import PCA
from scipy.stats import zscore
from pyod.models.mad import MAD

sys.path.append("../utils/")
from myFunctions import jsonGzipToDataframe, toDommyColumns

Extraemos los datos del archivo origina y lo cargamos en un dataframe

In [ ]:
df = jsonGzipToDataframe('../datasource/steam_games.json.gz')

In [ ]:
df.info()

Nos aseguramos de trabajar en otro espacio de memoria para no tener que volver a cargar el JSON

In [47]:
# Nos aseguramos de crear un nuevo objeto en memoria usando 'copy()'
dfSteamGames = df.copy()

Si hay 'id' duplicados nos quedamos con el primero y los demas los eliminamos

In [48]:
# Eliminar filas duplicadas basadas en la columna 'user_id'
dfSteamGames = dfSteamGames.drop_duplicates(['id'], keep = 'first').reset_index(drop=True)

In [65]:
dfSteamGames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32125 entries, 0 to 32124
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24078 non-null  object 
 1   genres        28844 non-null  object 
 2   app_name      32124 non-null  object 
 3   title         30076 non-null  object 
 4   url           32125 non-null  object 
 5   release_date  30059 non-null  object 
 6   tags          31963 non-null  object 
 7   reviews_url   32125 non-null  object 
 8   specs         31456 non-null  object 
 9   price         30748 non-null  float64
 10  early_access  32125 non-null  object 
 11  id            32125 non-null  object 
 12  developer     28828 non-null  object 
dtypes: float64(1), object(12)
memory usage: 3.2+ MB


In [ ]:
# Aplicamos una mascara para marcar los valores NaN y los sumamos por columna
dfSteamGames.isna().sum()

In [49]:
# Elimina las filas que tienen valor NaN en la columna 'id'
dfSteamGames = dfSteamGames.dropna(subset=['id'])

In [66]:
dfSteamGames['id'].unique()

array(['761140', '643980', '670290', ..., '610660', '658870', '681550'],
      dtype=object)

In [51]:
# Crear una lista con los valores a eliminar
invalidValues = ['Free Movie', 'Install Now', 'Install Theme', 'Third-party', 'Play Now']

In [52]:
# # Crear una lista con los valores a reemplazar
replaceValues = ['Free', 'Free Demo', 'Free Mod', 'Free to Use', 'Free To Play', 'Free to Play',
   'Free to Try', 'Play the Demo', 'Play for Free!', 'Free HITMAN™ Holiday Pack',
   'Play WARMACHINE: Tactics Demo']

¿Son mucho o pocos los valores invalidos en el precio?

In [53]:
dfSteamGames['price'].value_counts()[invalidValues]

price
Free Movie       1
Install Now      1
Install Theme    1
Third-party      2
Play Now         2
Name: count, dtype: int64

In [54]:
# Eliminar las filas que tengan los valores invalidos en la columna 'price'
dfSteamGames = dfSteamGames.loc[~dfSteamGames['price'].isin(invalidValues)].reset_index(drop = True)

In [55]:
# Sustituir el valor 'Free' por 0.00 en la columna 'price'
dfSteamGames['price'].replace(replaceValues, 0.00, inplace = True)

dfSteamGames['price'].replace(['Starting at $499.00'], 499.00, inplace = True)
dfSteamGames['price'].replace(['Starting at $449.00'], 449.00, inplace = True)

In [62]:
dfSteamGames['price'].unique()

array([4.9900e+00, 0.0000e+00, 9.9000e-01, 2.9900e+00, 3.9900e+00,
       9.9900e+00, 1.8990e+01, 2.9990e+01,        nan, 1.0990e+01,
       1.5900e+00, 1.4990e+01, 1.9900e+00, 5.9990e+01, 8.9900e+00,
       6.9900e+00, 7.9900e+00, 3.9990e+01, 1.9990e+01, 7.4900e+00,
       1.2990e+01, 5.9900e+00, 2.4900e+00, 1.5990e+01, 1.2500e+00,
       2.4990e+01, 1.7990e+01, 6.1990e+01, 3.4900e+00, 1.1990e+01,
       1.3990e+01, 3.4990e+01, 7.4760e+01, 1.4900e+00, 3.2990e+01,
       9.9990e+01, 1.4950e+01, 6.9990e+01, 1.6990e+01, 7.9990e+01,
       4.9990e+01, 5.0000e+00, 4.4990e+01, 1.3980e+01, 2.9960e+01,
       1.1999e+02, 1.0999e+02, 1.4999e+02, 7.7171e+02, 2.1990e+01,
       8.9990e+01, 9.8000e-01, 1.3992e+02, 4.2900e+00, 6.4990e+01,
       5.4990e+01, 7.4990e+01, 8.9000e-01, 5.0000e-01, 2.9999e+02,
       1.2900e+00, 3.0000e+00, 1.5000e+01, 5.4900e+00, 2.3990e+01,
       4.9000e+01, 2.0990e+01, 1.0930e+01, 1.3900e+00, 3.6990e+01,
       4.4900e+00, 2.0000e+00, 4.0000e+00, 9.0000e+00, 2.3499e

In [64]:
dfSteamGames['price'].value_counts()

price
4.99      4278
9.99      3902
2.99      3429
0.99      2607
1.99      2541
          ... 
26.99        1
179.00       1
10.49        1
6.66         1
160.91       1
Name: count, Length: 147, dtype: int64

In [60]:
dfSteamGames['price'] = dfSteamGames['price'].round(2)

In [68]:
dfSteamGames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32125 entries, 0 to 32124
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24078 non-null  object 
 1   genres        28844 non-null  object 
 2   app_name      32124 non-null  object 
 3   title         30076 non-null  object 
 4   url           32125 non-null  object 
 5   release_date  30059 non-null  object 
 6   tags          31963 non-null  object 
 7   reviews_url   32125 non-null  object 
 8   specs         31456 non-null  object 
 9   price         30748 non-null  float64
 10  early_access  32125 non-null  object 
 11  id            32125 non-null  int64  
 12  developer     28828 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 3.2+ MB


Eliminemos los juegos que no tienen 'id'

In [ ]:
# Eliminar las filas que contienen valores NaN en la columna 'id'
dfSteamGames = dfSteamGames.dropna(subset = ['id']).reset_index(drop = True)

La columna 'price' debe ser de tipo decimal... cambiemosla a Float

In [67]:
# Convertir la columna 'price' a tipo float
dfSteamGames['id'] = dfSteamGames['id'].astype(int)

Los NaN son de tipo float
¿Cuánto representarán del total de valores en la columna 'tags'?

In [ ]:
# Crear un nuevo DataFrame solo con las filas que tienen tipo string y no NaN Float en la columna
# 'tags'
dfSinTagsNaN = df[df['tags'].apply(lambda x: isinstance(x, str))]
dfConTagsNaN = df[df['tags'].apply(lambda x: isinstance(x, float))]
print('String son', dfSinTagsNaN.shape[0])
print('NaN son', dfConTagsNaN.shape[0])

Entonces nos quedearemos con los juegos cuyos 'tags' no sea uno de esos NaN Float

In [ ]:
df = dfSinTagsNaN.reset_index(drop=True)
df.info()

Obtengamos las columnas dummies de la columna 'tags'

In [ ]:
# Pero antes sustituyamos los NaN de la columna 'tags' por listas vacias
df['tags'] = df['tags'].fillna('[]')

# Obtengamos las columnas dummies de la columna 'tags'
dummyColumnsTags = toDommyColumns(df,'tags')

# Seamos consistentes con los nombres de columnas
dummyColumnsTags.columns = dummyColumnsTags.columns.str.lower().str.replace(' ', '_') 

In [ ]:
dummyColumnsTags.shape

Veamos cuáles son las columnas dummies más valiosas de la columna 'tags'

In [ ]:
print(dummyColumnsTags.sum(axis=0).sort_values(ascending=False).head(30) / dummyColumnsTags.shape[0])
#dummyColumnsTags.sum().sort_values(ascending = False).head(20) / dummyColumnsTags.shape[0]

In [ ]:
# Sumemos los valores de la columna 'tags', los disponemos en orden descendente, seleccionamos los
# 20 valores más altos y luego los normalizamos dividiéndolos por la longitud del DataFrame original
dummyColumnsTags.sum().sort_values(ascending = False).head(20) / dummyColumnsTags.shape[0]

Nos quedaremos con las siguientes columnas

In [ ]:
tagsColumns = ['tags_indie', 'tags_action', 'tags_adventure', 'tags_casual', 'tags_simulation']
tagsColumns += ['tags_strategy', 'tags_rpg', 'tags_singleplayer', 'tags_free_to_play', 'tags_multiplayer']
dummyTags = dummyColumnsTags[tagsColumns]

In [ ]:
dummyTags.info()

Es momento de unir las columnas dummies de tags al dataframe original

In [ ]:
# Insertemos las columnas dummies al dataframe original
df = pd.concat([df, dummyTags], axis = 1)

In [ ]:
df.head()

In [ ]:
df.info()

Los NaN son de tipo float
¿Cuánto representarán del total de valores en la columna 'genres'?

In [ ]:
# Crear un nuevo DataFrame solo con las filas que tienen tipo string y no NaN Float en la columna
# 'genres'
dfSinGenresNaN = df[df['genres'].apply(lambda x: isinstance(x, str))]
dfConGenresNaN = df[df['genres'].apply(lambda x: isinstance(x, float))]
print('String son', dfSinGenresNaN.shape[0])
print('NaN son', dfConGenresNaN.shape[0])

Entonces nos quedearemos con los juegos cuyos 'genres' no sea uno de esos NaN Float

In [ ]:
df = dfSinGenresNaN.reset_index(drop = True)
df.info()

Obtengamos las columnas dummies de la columna 'genres'

In [ ]:
# Obtengamos las columnas dummies de la columna 'genres'
dummyColumnsGenres = toDommyColumns(df, 'genres')

# Seamos consistentes con los nombres de columnas
dummyColumnsGenres.columns = dummyColumnsGenres.columns.str.lower().str.replace(' ', '_') 

In [ ]:
# Sumemos los valores de la columna 'genres', los ordenamos en orden descendente, seleccionamos los 20
# valores más altos y luego los normalizamos dividiéndolos por la longitud del DataFrame original
dummyColumnsGenres.sum().sort_values(ascending = False).head(20) / len(dummyColumnsGenres)

Nos quedaremos con la siguientes columnas

In [ ]:
genresColumns = ['genres_indie', 'genres_action', 'genres_casual', 'genres_adventure']
genresColumns += ['genres_strategy', 'genres_simulation', 'genres_rpg', 'genres_free_to_play']
genresColumns += ['genres_early_access', 'genres_sports', 'genres_massively_multiplayer', 'genres_racing']
dummyGenres = dummyColumnsGenres[genresColumns]

In [ ]:
#dummyGenres

In [ ]:
#df.shape

Es momento de unir las columnas dummies de genres al dataframe original

In [ ]:
# Insertemos las columnas dummies al dataframe original
df = pd.concat([df, dummyGenres], axis = 1)

In [ ]:
df.iloc[:, 8:9]

In [ ]:
df.head()
#df.loc[df['title'] == 'Lost Summoner Kitty']

A crear la columna 'year'

In [ ]:
# Extraer el año de la columna 'release_date' y crear la columna 'year'
df['year'] = df['release_date'].str.extract(r'(\d{4})')

In [ ]:
df['year'].unique()

In [ ]:
df.info()

Ok, está quedando mejor. Parece que las columnas 'url' y 'reviews_url' sobran porque no aportan nada
al análisis, vamos a quitarlas


In [ ]:
# Eliminar las columnas 'url' y 'reviews_url'
df = df.drop(['url', 'reviews_url'], axis=1)

Ya le extrajimos el año a la columna 'release_date', vamos a borrarla. Es redundante tener a
'genres' y 'tags' porque las desplegamos como dummies, tambien a borrarlas. 

In [ ]:
# Eliminar las columnas 'release_date', 'genres' y 'tags'
df = df.drop(['release_date', 'genres', 'tags'], axis=1)

In [ ]:
df.head()

Los True y False de la columna 'early_access' mejor los cambiamos a unos y ceros respectivamente

In [ ]:
# Cambiar la columna 'early_access' a tipo entero en el DataFrame 'df'
df['early_access'] = df['early_access'].astype(int)

In [ ]:
# Ver el contenido de la columna 'tags' de la primera fila
type(df.iloc[0]['early_access'])

In [ ]:
df.info()

Busquemos pistas sobre la presencia de outliers con un histograma

In [ ]:
# Create a histogram using Seaborn
g = sns.histplot(data = df, x = 'price')
# Add labels
g.set_xlabel('Total price per game')

En el histograma podemos ver que los datos se concentran por debajo de los 50$ aproximadamente

Busquemos pistas sobre la presencia de outliers con un diagrama de caja

In [ ]:
# Create a box plot
g = sns.boxplot(data = df, x = 'price')

# Add a title and change xlabel
g.set_title('Box Plot of Total')
g.set_xlabel('Total price per game')

En el gráfico de caja podemos ver claramente la presencia de outliers en la columna ‘price’,

Ataquemos ahora con Z-Score

In [ ]:
# Calculate z-score for each data point and compute its absolute value
z_scores = zscore(df['price'])
abs_z_scores = np.abs(z_scores)

# Select the outliers using a threshold of 3
outliers = df[abs_z_scores > 3]
outliers.head()

In [ ]:
# Obtain number of outliers
print(f'Number of outliers: {len(outliers)}')

Pero no podemos fiarnos de estos 287 porque el método z-score sólo es apropiada para distribuciones normales. Como vimos en el histograma, los datos para la variable ‘price’ están sesgados hacia la derecha por lo que debemos afinar la puntería.

Busquemos pistas sobre la presencia de outliers con MAD-Z-Score

In [ ]:
# Set threshold to 3.5
mad = MAD(threshold = 3.5)

# Convert the 'total' column into a 2D numpy array
priceReshaped = df['price'].values.reshape(-1, 1)

# Generate inline and outlier labels
labels = mad.fit(priceReshaped).labels_
labels

In [ ]:
# Obtain number of outliers
print(f'Number of outliers: {labels.sum()}')